In [1]:
import json
import requests
import pickle
from time import sleep
from datetime import datetime
from collections import defaultdict
import calendar

In [2]:
cred = ('user', 'password')
fmt = '%Y-%m-%d'

#tags we want http://docs.intrinio.com/tags/intrinio-public#home
tags = {
    "PE":"pricetoearnings",
    "EPS": "basiceps",
    "PBV": "pricetobook",
    "NM": "profitmargin",
    "NINR": "nwctorev"
}

#ticks we want (DOW JONES)
dowticks = ["MMM","AAPL","BA","CAT","CVX"]#,"CSCO","KO","DD","XOM","GE","HD","IBM","INTC"#,"JNJ","JPM","MCD","MRK","NKE","PFE","PG","TRV","UNH","UTX","VZ","V","WMT","DIS"]
#["AXP"]
output = defaultdict(dict)
quart = {}

In [9]:
#download all the data and save it as output
for tick in dowticks:
    output[tick] = defaultdict(dict)
    quart[tick] = defaultdict(list)
    print "running for",tick
    for tagn, tagq in tags.items():
        sleep(.5)
        r = requests.get("https://api.intrinio.com/historical_data?identifier=%s&item=%s" %(tick,tagq), auth=cred)
        tickrawdata = json.loads(r.text)
        
        #convert to unix timestamp
        tickdate = [(datetime.strptime(i["date"],fmt).strftime("%s") , i["value"]) for i in tickrawdata["data"]]
        tickrawlen = len((tickrawdata["data"]))
        
        #save data with the [tag][date]
        for date in tickdate:
            if date[0] not in output[tick]:
                output[tick][date[0]] = {}
            
            #get a dates of quarterly
            if tickrawlen < 100:
                quart[tick][tagn].append(date[0]) 
            output[tick][date[0]][tagn] = date[1]

pickle.dump( output, open( "save.p", "wb" ) )

running for MMM
31
2335
31
31
2335
running for AAPL
32
2399
32
32
2399
running for BA
31
2335
31
31
2335
running for CAT
31
2335
31
31
2335
running for CVX
31
2335
31
31
2335


In [16]:
output = pickle.load( open( "save.p", "rb" ) )

In [17]:
#give quarterly data to all the data 
proc = dict(output)

for tick in dowticks:
    for k in quart[tick].keys():
        quart[tick][k] = [int(v) for v in quart[tick][k]]
        for d1,d2 in proc[tick].items():
            if len(d2.values()) <= 4:
                try:
                    neardate = str(min([i for i in quart[tick][k] if i <= int(d1)], key=lambda x:abs(x- int(d1))))
                    proc[tick][d1][k] = proc[tick][neardate][k]
                except ValueError:
                    print "error at", d1

error at 1190952000
error at 1190952000
error at 1190952000


In [18]:
#remove old years
for tick in dowticks:
    for k,v in proc[tick].items():
            if datetime.fromtimestamp(int(k)).strftime('%Y') in [2007,2008]:
                del proc[tick][k]

In [19]:
#remove data that doesn't have all 5 values , should be only sat,sun
for tick in dowticks:
    for k,v in proc[tick].items():
        if len(v) != 5:
            print datetime.fromtimestamp(int(k)).strftime('%a')
            del proc[tick][k]

Sat
Sat
Sat
Sun
Sun
Sun
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Fri
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sat
Sun
Sun
Sun
Sat
Sat
Sat
Sat
Sun
Sun
Sun
Sat
Sat
Sat
Sat
Sun
Sun
Sun
Sat


In [20]:
#exported cleaned data
pickle.dump(dict(proc), open( "proc.p", "wb" ) )